In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks', color_codes=True)

from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

/projects/rlmolecule/pstjohn/envs/tf2_gpu/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
cdf_spin = pd.read_csv('/projects/rlmolecule/pstjohn/atom_spins/cdf_spins.csv.gz')
cdf_bv = pd.read_csv('/projects/rlmolecule/pstjohn/atom_spins/cdf_buried_volume.csv.gz', index_col=0)
cdf = cdf_spin.merge(cdf_bv, on=['smiles', 'atom_index'], how='left')
cdf.head()

/projects/rlmolecule/pstjohn/envs/tf2_gpu/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,smiles,atom_type,atom_index,spin,buried_vol
0,[Br],Br,0,1.000000,NaN
1,BrC12C3C4[C]5C3C1C5C42,Br,0,0.013204,31.048787
2,BrC12C3C4[C]5C3C1C5C42,C,1,-0.014127,56.750489
3,BrC12C3C4[C]5C3C1C5C42,C,2,0.076213,50.270936
4,BrC12C3C4[C]5C3C1C5C42,C,3,-0.102171,46.228361


In [4]:
import psycopg2

dbparams = {
    'dbname': 'bde',
    'port': 5432,
    'host': 'yuma.hpc.nrel.gov',
    'user': 'bdeops',
    'password': 'BD30ps99!',
    'options': f'-c search_path=bde',
}

with psycopg2.connect(**dbparams) as conn:

    redf = pd.read_sql_query("""
    SELECT * from redoxcompound where status = 'finished'
    """, conn)    

In [5]:
radical = redf[redf.estate == 'radical'].set_index('smiles')
oxidized = redf[redf.estate == 'oxidized'].set_index('smiles')
reduced = redf[redf.estate == 'reduced'].set_index('smiles')

ionization = (oxidized.enthalpy - radical.enthalpy).dropna()
eas = (radical.enthalpy - reduced.enthalpy).dropna()

ionization.name = 'ionization energy'
eas.name = 'electron affinity'

merged_redox = pd.DataFrame(ionization).merge(pd.DataFrame(eas), left_index=True, right_index=True, how='outer')

In [6]:
redf_redox_df = ((merged_redox * 627.509) / 23.061 - 4.281).reset_index()
redf_redox_df.to_csv('/projects/rlmolecule/pstjohn/atom_spins/redf_redox.csv.gz', compression='gzip', index=False)

In [7]:
redf_redox_df

,smiles,ionization energy,electron affinity
0,C#C/C(C)=C(\C)[C@@H]([CH2])C,NaN,-1.429495
1,C#C/C(C)=C/C([CH2])=C,0.973058,-1.152652
2,C#C/C(C)=C/CCC([CH2])=O,1.370200,0.133522
3,C#C/C(C)=C/CC[C]=O,-0.359756,-1.147401
4,C#C/C(C)=C/C[C@H]([CH2])CO,0.445249,NaN
...,...,...,...
14190,c1ccc([C]2CCO2)cc1,-0.399865,-1.653685
14191,c1cnc2c(c1)=N[N]N=2,2.769816,1.722091
14192,c1cncc([C]2CCN2)c1,-0.936625,-1.697794
14193,c1coc([C]2OCCO2)c1,-0.815292,-1.481577


In [2]:
redf_spin = pd.read_csv('/projects/rlmolecule/pstjohn/atom_spins/redf_spins.csv.gz')
redf_bv = pd.read_csv('/projects/rlmolecule/pstjohn/atom_spins/redf_buried_volume.csv.gz')
redf = redf_spin.merge(redf_bv, on=['smiles', 'atom_index', 'atom_type'], how='left')

redf_redox = pd.read_csv('/projects/rlmolecule/pstjohn/atom_spins/redf_redox.csv.gz')
redf_redox = redf_redox.set_index('smiles').reindex(redf.smiles.unique())

In [3]:
df = redf

In [5]:
for smiles, idf in tqdm(df.groupby('smiles')):
#    input_dict = preprocessor.construct_feature_matrices(smiles, train=train)
    input_dict = {}
    spin = idf.set_index('atom_index').sort_index().spin
    fractional_spin = spin.abs() / spin.abs().sum()
    input_dict['spin'] = fractional_spin.values
    input_dict['bur_vol'] = idf.bur_vol.values
    input_dict['redox'] = redf_redox.loc[smiles].values
    
    
    break

  0%|          | 0/14599 [00:00<?, ?it/s]


In [7]:
redf_redox.loc[smiles]

ionization energy          NaN
electron affinity    65.758551
Name: C#C/C(C)=C(\C)[C@@H]([CH2])C, dtype: float64

In [6]:
input_dict

{'spin': array([2.51354704e-04, 2.16834236e-03, 2.72572958e-03, 2.69471829e-03,
        1.47687210e-02, 2.15202063e-03, 9.29849187e-03, 8.41216459e-01,
        1.19222106e-02, 3.10112946e-05, 2.02144676e-03, 6.02271985e-04,
        5.69628517e-04, 3.90905530e-04, 3.19089900e-03, 1.19801528e-03,
        4.23973363e-02, 3.08692956e-02, 2.79166939e-02, 1.46895606e-03,
        2.05409023e-03, 9.14017105e-05]),
 'bur_vol': array([27.5506119 , 40.60638847, 51.73196462, 35.91999521, 57.65526653,
        41.53958628, 50.91490544, 38.62808771, 37.46743364,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan]),
 'redox': array([        nan, 65.75855064])}

,ionization energy,electron affinity
smiles,,
[CH2]C1=COCC1,95.084556,61.300727
CC[C@H](C)C[CH]CCO,106.160718,NaN
CS[C@H]1CC[C@@H]1NC1[CH]C1,NaN,75.592244
COC(C)[O],NaN,117.022898
[CH2]CCCCOC/C=C/C,NaN,62.398240
...,...,...
C=CC[CH]C=C(C)C,99.745693,58.719155
[CH2][C@@H]1CCC[C@@H](C)C1,NaN,60.178741
C=CO[C@H](C)[C](C)C,97.836811,60.549598


In [45]:
redf_redox

,smiles,ionization energy,electron affinity
0,C#C/C(C)=C(\C)[C@@H]([CH2])C,NaN,65.758551
1,C#C/C(C)=C/C([CH2])=C,121.163830,72.142827
2,C#C/C(C)=C/CCC([CH2])=O,130.322324,101.803295
3,C#C/C(C)=C/CC[C]=O,90.427812,72.263936
4,C#C/C(C)=C/C[C@H]([CH2])CO,108.992038,NaN
...,...,...,...
14190,c1ccc([C]2CCO2)cc1,89.502864,60.588504
14191,c1cnc2c(c1)=N[N]N=2,162.598877,138.437271
14192,c1cncc([C]2CCN2)c1,77.124621,59.571312
14193,c1coc([C]2OCCO2)c1,79.922684,64.557498


In [43]:
smiles

'C#C/C(C)=C(\\C)[C@@H]([CH2])C'

In [55]:
!nvidia-smi

Wed Aug 26 08:39:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:37:00.0 Off |                  Off |
| 31%   43C    P2    36W / 250W |   8426MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  